# Introduction

This tutorial illustrates how to import an *ObjTables* dataset from a file, export an *ObjTables* dataset to a file, convert an *ObjTables* dataset between two supported file formats, and pretty print a file that contains an *ObjTables* dataset. The tutorial uses an address book of CEOs as an example.

##### Supported file formats
*ObjTables* supports seven file formats:
* Excel workbook
* Collection of comma-separated values (CSV) files
* Collection of tab-separated values (TSV) files
* Single text file which contains multiple comma-separated tables (MULTI.CSV)
* Single text file which contains multiple tab-separated tables (MULTI.TSV)
* JavaScript Object Notation (JSON) file
* YAML Ain't Markup Language (YAML) file

##### Use cases for each file format
* **View or edit a dataset**: We recommend using an Excel workbook as a graphical user interface for viewing or editing a dataset. *ObjTables* leverages several Excel features:
  * Optionally includes additional worksheets that contain a table of contents and the schema for the dataset.
  * Highlights, freezes, and protects the row/column headings of each table.
  * Uses comments to provide inline help for each row/column.
  * Provides drop-down menus for each enumeration and \*-to-one relationship.
  * Uses Excel validation to help users quickly find and correct errors.
  * Hides all unused rows and columns.
  
* **Share or publish a dataset**: We recommend using Excel workbooks to share and publish datasets because an entire dataset and its schema can be captured by a single file and because Excel workbooks are easy for humans to read.

* **Revision a dataset**: We recommend using collections of CSV or TSV files to revision datasets with version control systems such as Git. Version control systems can easily difference and merge CSV and TSV files.

* **Import a dataset into another programming language or export a dataset from another language**: We recommend using JSON files to work with *ObjTables* datasets in other languages because most languages have methods for parsing JSON files.

# Define a schema for an address book

First, as described in [Tutorial 1](1.%20Building%20and%20visualizing%20schemas.ipynb), use *ObjTables* to define a schema for an address book.

In [1]:
import enum
import obj_tables
import types


# Define classes to represent companies, their CEOs, and their addresses
class Address(obj_tables.Model):
    street = obj_tables.StringAttribute(unique=True, primary=True, verbose_name='Street')
    city = obj_tables.StringAttribute(verbose_name='City')
    state = obj_tables.StringAttribute(verbose_name='State')
    zip_code = obj_tables.StringAttribute(verbose_name='Zip code')
    country = obj_tables.StringAttribute(verbose_name='Country')

    class Meta(obj_tables.Model.Meta):
        table_format = obj_tables.TableFormat.multiple_cells
        attribute_order = ('street', 'city', 'state', 'zip_code', 'country',)
        verbose_name = 'Address'
        verbose_name_plural = 'Addresses'
        

class Company(obj_tables.Model):
    name = obj_tables.StringAttribute(unique=True, primary=True, verbose_name='Name')
    url = obj_tables.UrlAttribute(verbose_name='URL')
    address = obj_tables.OneToOneAttribute(Address, related_name='company', verbose_name='Address')

    class Meta(obj_tables.Model.Meta):
        table_format = obj_tables.TableFormat.column
        attribute_order = ('name', 'url', 'address',)
        verbose_name = 'Company'
        verbose_name_plural = 'Companies'


class PersonType(str, enum.Enum):
    family = 'family'
    friend = 'friend'
    business = 'business'


class Person(obj_tables.Model):
    name = obj_tables.StringAttribute(unique=True, primary=True, verbose_name='Name')
    type = obj_tables.EnumAttribute(PersonType, verbose_name='Type')
    company = obj_tables.ManyToOneAttribute(Company, related_name='employees', verbose_name='Company')
    email_address = obj_tables.EmailAttribute(verbose_name='Email address')
    phone_number = obj_tables.StringAttribute(verbose_name='Phone number')
    address = obj_tables.ManyToOneAttribute(Address, related_name='people', verbose_name='Address')

    class Meta(obj_tables.Model.Meta):
        table_format = obj_tables.TableFormat.row
        attribute_order = ('name', 'type', 'company', 'email_address', 'phone_number', 'address',)
        verbose_name = 'Person'
        verbose_name_plural = 'People'
        

# Add the classes to a module
schema = type('address_book', (types.ModuleType, ), {
    'Address': Address,
    'Company': Company,
    'Person': Person,
    'PersonType': PersonType,
    'models': [Company, Person],
})

# Create an adress book of technology companies and their CEOs

Use the address book schema to build an address book of technology companies and their CEOs.

In [2]:
# Tim Cook of Apple
apple = Company(name='Apple',
                url='https://www.apple.com/',
                address=Address(street='10600 N Tantau Ave',
                                city='Cupertino',
                                state='CA',
                                zip_code='95014',
                                country='US'))
cook = Person(name='Tim Cook',
              type=PersonType.business,
              company=apple,
              email_address='tcook@apple.com',
              phone_number='408-996-1010',
              address=apple.address)

# Reed Hasting of Netflix
netflix = Company(name='Netflix',
                  url='https://www.netflix.com/',
                  address=Address(street='100 Winchester Cir',
                                  city='Los Gatos',
                                  state='CA',
                                  zip_code='95032',
                                  country='US'))
hastings = Person(name='Reed Hastings',
                  type=PersonType.business,
                  company=netflix,
                  email_address='reed.hastings@netflix.com',
                  phone_number='408-540-3700',
                  address=netflix.address)

# Sundar Pichai of Google
google = Company(name='Google',
                 url='https://www.google.com/',
                 address=Address(street='1600 Amphitheatre Pkwy',
                                 city='Mountain View',
                                 state='CA',
                                 zip_code='94043',
                                 country='US'))
pichai = Person(name='Sundar Pichai',
                type=PersonType.business,
                company=google,
                email_address='sundar@google.com',
                phone_number='650-253-0000',
                address=google.address)

# Mark Zuckerberg of Facebook
facebook = Company(name='Facebook',
                   url='https://www.facebook.com/',
                   address=Address(street='1 Hacker Way #15',
                                   city='Menlo Park',
                                   state='CA',
                                   zip_code='94025',
                                   country='US'))
zuckerberg = Person(name='Mark Zuckerberg',
                    type=PersonType.business,
                    company=facebook,
                    email_address='zuck@fb.com',
                    phone_number='650-543-4800',
                    address=facebook.address)

# Merge the companies and CEOs into a single address book
companies = [apple, facebook, google, netflix]
ceos = [zuckerberg, hastings, pichai, cook]
address_book = companies + ceos
address_book_by_class = {
    Company: companies,
    Person: ceos,
}

# Export the address book to the supported file formats

##### Import the `obj_tables.io` module

In [3]:
import obj_tables.io

##### Export the address book to Excel, CSV, TSV, JSON, and YAML files
Use `obj_tables.io.Writer` to export the address book to Excel, CSV, TSV, JSON, and YAML files.

In [4]:
import os
import tempfile
from IPython.core.display import display, HTML

dir = 'Address book'
xlsx_file = os.path.join(dir, 'Address book.xlsx')
csv_file = os.path.join(dir, 'Address book.csv/*.csv')
tsv_file = os.path.join(dir, 'Address book.tsv/*.tsv')
multi_csv_file = os.path.join(dir, 'Address book.multi.csv')
multi_tsv_file = os.path.join(dir, 'Address book.multi.tsv')
json_file = os.path.join(dir, 'Address book.json')
yml_file = os.path.join(dir, 'Address book.yml')

if not os.path.isdir(dir):
    os.makedirs(dir)

if not os.path.isdir(os.path.join(dir, 'Address book.csv')):
    os.makedirs(os.path.join(dir, 'Address book.csv'))
    
if not os.path.isdir(os.path.join(dir, 'Address book.tsv')):
    os.makedirs(os.path.join(dir, 'Address book.tsv'))
    
# export address book
for file in [xlsx_file, csv_file, tsv_file, multi_csv_file, multi_tsv_file, json_file, yml_file]:
    obj_tables.io.Writer().run(file, address_book, models=schema.models, 
                               write_toc=True,
                               write_schema=True)

# display links to generated files
display(
    HTML(
        "<ul>"
        + "<li><a target='_blank' href='{}'>Excel workbook</a></li>".format(xlsx_file)
        + "<li><a target='_blank' href='{}'>CSV files</a></li>".format(os.path.dirname(csv_file))
        + "<li><a target='_blank' href='{}'>TSV files</a></li>".format(os.path.dirname(tsv_file))
        + "<li><a target='_blank' href='{}'>MULTI.CSV file</a></li>".format(multi_csv_file)
        + "<li><a target='_blank' href='{}'>MULTI.TSV file</a></li>".format(multi_tsv_file)
        + "<li><a target='_blank' href='{}'>JSON file</a></li>".format(json_file)
        + "<li><a target='_blank' href='{}'>YAML file</a></li>".format(yml_file)
        + "</ul>"
    )
)

# Import the address book from the supported file formats

Use `obj_tables.io.Reader` to import the address book from Excel, CSV, TSV, JSON, and YAML files.

In [5]:
for file in [xlsx_file, csv_file, tsv_file, multi_csv_file, multi_tsv_file, json_file, yml_file]:
    # Read the address book
    address_book_by_class_copy = obj_tables.io.Reader().run(file,
                                                            models=schema.models)

    # Check that the imported address book is equal to the original
    for cls in address_book_by_class.keys():    
        objs = sorted(address_book_by_class[cls], key=lambda obj: obj.name)
        copies = sorted(address_book_by_class_copy[cls], key=lambda obj: obj.name)
        for obj, copy in zip(objs, copies):
            assert obj.is_equal(copy)

#  Convert the address book between the supported file formats

Use `obj_tables.io.convert` to convert the address book among Excel, CSV, TSV, JSON, and YAML files.

In [6]:
# Read the address book
obj_tables.io.convert(xlsx_file, csv_file, schema_name='Address book', models=schema.models)

# Check that the converted address book is equal to the original
address_book_by_class_copy = obj_tables.io.Reader().run(csv_file,
                                                        schema_name='Address book',
                                                        models=schema.models)

for cls in address_book_by_class.keys():    
    for obj, copy in zip(address_book_by_class[cls], address_book_by_class_copy[cls]):
        assert obj.is_equal(copy)

# Pretty print an Excel file which contains an address book

`obj_tables.io.convert` can also be used to pretty print files. This operation makes the following changes to files:
  * Orders the worksheets and columns.
  * Removes extraneous worksheets and columns that are not controlled by the schema.
  * Additional changes to Excel files:
    * Optionally includes additional worksheets that contain a table of contents and the schema for the dataset.  
    * Highlights, freezes, and protects the column headings of each table.  
    * Uses comments to provide inline help for each column.
    * Provides drop-down menus for each enumeration and \*-to-one relationship.
    * Uses Excel validation to help users quickly find and correct errors.
    * Hides all unused rows and columns.

In [7]:
# Pretty print the address book
obj_tables.io.convert(xlsx_file, xlsx_file, schema_name='Address book', models=schema.models)

# Check that the pretty printed address book is equal to the original
address_book_by_class_copy = obj_tables.io.Reader().run(xlsx_file,
                                                        schema_name='Address book',
                                                        models=schema.models)

for cls in address_book_by_class.keys():    
    for obj, copy in zip(address_book_by_class[cls], address_book_by_class_copy[cls]):
        assert obj.is_equal(copy)